<a href="https://colab.research.google.com/github/yesstarr/2024k-hackerton_COSMICA/blob/main/ai_model_MFshortscore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import json
from math import radians, cos, sin, sqrt, atan2
from itertools import permutations

# Haversine 함수를 사용해 두 좌표 사이의 거리 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Brute-force TSP 경로 최적화
def tsp_bruteforce(route_coords):
    num_points = len(route_coords)
    all_routes = permutations(range(num_points))
    min_distance = float('inf')

    for route in all_routes:
        current_distance = 0
        for i in range(len(route) - 1):
            current_distance += haversine(*route_coords[route[i]], *route_coords[route[i + 1]])
        current_distance += haversine(*route_coords[route[-1]], *route_coords[route[0]])  # 시작점으로 돌아가기

        if current_distance < min_distance:
            min_distance = current_distance

    return min_distance

# Matrix Factorization
class MF():
    def __init__(self, R, hyper_params):
        self.R = np.array(R)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']

        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, cols = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, cols)]

    def rmse(self):
        xs, ys = self.R.nonzero()
        error = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            error.append(self.R[x, y] - prediction)
        return np.sqrt(np.mean(np.array(error) ** 2))

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def train(self):
        for it in range(self.iterations):
            np.random.shuffle(self.samples)
            for i, j, r in self.samples:
                prediction = self.get_prediction(i, j)
                e = r - prediction

                self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
                self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

                self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
                self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

            rmse = self.rmse()

    def recommend(self):
        predictions = np.zeros((self.num_users, self.num_items))
        for i in range(self.num_users):
            for j in range(self.num_items):
                predictions[i, j] = self.get_prediction(i, j)
        return predictions

# 경로 데이터를 평가 매트릭스로 변환
json_data_list = [
    '''
    {
      "data" : [
         {"lat" :"36.632473380701","long" : "127.45314301376"},
         {"lat" :"36.632473380705","long" : "127.45314301387"},
         {"lat" :"36.632473380706","long" : "127.45314301398"},
         {"lat" :"36.632473380707","long" : "127.45314301399"},
         {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380705","long" : "127.45314301377"},
          {"lat" :"36.632473380706","long" : "127.45314301378"},
          {"lat" :"36.632473380707","long" : "127.45314301379"},
          {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    '''
]

# 각 경로의 최단 거리 계산
distances = []
for json_data in json_data_list:
    data = json.loads(json_data)['data']
    coords = [(float(point['lat']), float(point['long'])) for point in data]
    min_distance = tsp_bruteforce(coords)
    distances.append(min_distance)

# 거리를 점수로 변환하여 평가 매트릭스 생성
R = [[3 if i == j else max(5 - distances[i] / max(distances) * 5, 1) for j in range(len(distances))] for i in range(len(distances))]

# 하이퍼파라미터 설정 및 MF 모델 생성
hyper_params = {'K': 10, 'alpha': 0.01, 'beta': 0.01, 'iterations': 600, 'verbose': True}
mf = MF(R, hyper_params)
mf.train()

# 경로 추천
predicted_scores = mf.recommend()
best_route_index = np.argmax(np.mean(predicted_scores, axis=0))
print(f"추천경로: {best_route_index + 1}번째 경로")
recommended_json_data = json_data_list[best_route_index]
print(recommended_json_data)


추천경로: 1번째 경로

    {
      "data" : [
         {"lat" :"36.632473380701","long" : "127.45314301376"},
         {"lat" :"36.632473380705","long" : "127.45314301387"},
         {"lat" :"36.632473380706","long" : "127.45314301398"},
         {"lat" :"36.632473380707","long" : "127.45314301399"},
         {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    
